In [46]:
from importlib import reload
import gpt2
reload(gpt2)
from gpt2 import GPT2LMHeadModel
from transformers import GPT2Tokenizer
import numpy as np
import pandas as pd
import json
import torch
from tqdm import tqdm

# Load the model

In [4]:
tokenizer = GPT2Tokenizer.from_pretrained("gpt2-xl")
model = GPT2LMHeadModel.from_pretrained("gpt2-xl")
model.eval()
print("") # To not output the string version of the whole model

# Qualitative Assessment

In [5]:
line = "The tiger licked its lips menacingly as it approached me. I felt"

input = tokenizer.encode(line.strip(), return_tensors="pt")

In [7]:
# This is where we decide which neurons to mask
with open('neurons_per_layer.json') as file:
    neurons_per_layer = json.load(file)

neurons_per_layer = {int(k):v for k, v in neurons_per_layer.items()}


In [238]:
normal_outputs = model.generate(
    input,
    max_length=30,
    do_sample=True,
    num_return_sequences=10,
    # no_repeat_ngram_size=2,
    # repetition_penalty=1.5,
    # top_p=0.92,
    # temperature=.85,
    # do_sample=True,
    # top_k=125,
    early_stopping=True,
    neurons_per_layer=None,
)

masked_outputs = model.generate(
    input,
    max_length=30,
    do_sample=True,
    num_return_sequences=10,
    # no_repeat_ngram_size=2,
    # repetition_penalty=1.5,
    # top_p=0.92,
    # temperature=.85,
    # do_sample=True,
    # top_k=125,
    early_stopping=True,
    neurons_per_layer=neurons_per_layer,
)


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


In [239]:
for output in normal_outputs:
    text = tokenizer.decode(output, skip_special_tokens=True)
    print(text, end='\n')
    print(143*'=')

The tiger licked its lips menacingly as it approached me. I felt a feeling of heat in all my dowsing rods. I thought,
The tiger licked its lips menacingly as it approached me. I felt my face slowly become red from embarrassment.

As soon as I was
The tiger licked its lips menacingly as it approached me. I felt very, very relaxed.

"What a curious creature, that tiger
The tiger licked its lips menacingly as it approached me. I felt goosebumps creep up my arms and neck and my stomach growling.
The tiger licked its lips menacingly as it approached me. I felt as though these two were destined for each other. If I were to fight
The tiger licked its lips menacingly as it approached me. I felt it pull back and raise its paws, bringing them up to me. My
The tiger licked its lips menacingly as it approached me. I felt the heat of its eyes, the hardness of its claws.

"
The tiger licked its lips menacingly as it approached me. I felt myself trembling as the pain in my arm grew.

It was then
The 

In [240]:
for output in masked_outputs:
    text = tokenizer.decode(output, skip_special_tokens=True)
    print(text, end='\n')
    print(143*'=')

The tiger licked its lips menacingly as it approached me. I felt fear and I had to show it that I was a serious martial artist.
The tiger licked its lips menacingly as it approached me. I felt a chill run down my spine. I had no idea what this thing could
The tiger licked its lips menacingly as it approached me. I felt as though my heart was going to beat out of my chest.


The tiger licked its lips menacingly as it approached me. I felt a slight tugging on my shirt, but the tiger continued its slow motion
The tiger licked its lips menacingly as it approached me. I felt an indescribable shock as its claws touched my face. The tiger
The tiger licked its lips menacingly as it approached me. I felt my heart start to beat faster as my body quivered. I was so
The tiger licked its lips menacingly as it approached me. I felt the need to draw from my current internal strength and not rely on the external
The tiger licked its lips menacingly as it approached me. I felt an immediate sense of dis

# Wasserstein Distance of Outputs

In [1]:
from scipy.stats import wasserstein_distance
from scipy.special import kl_div

In [41]:
line = "The tiger licked its lips menacingly as it approached me. I felt"

input = tokenizer.encode(line.strip(), return_tensors="pt")

In [42]:
with torch.no_grad():
    output = model(input)

In [43]:
input.shape

torch.Size([1, 15])

In [44]:
logits_probs = torch.nn.functional.softmax(output.logits, dim=0)

In [55]:
filename = "data/train_data_binary.csv"

df = pd.read_csv(filename)
neg_df = df[df.label == 0]
pos_df = df[df.label == 1]

neg_log_probs = []

for i, row in neg_df.iterrows():
    input = tokenizer.encode(row.sentence.strip(), return_tensors='pt')
    with torch.no_grad():
        output = model(input)
    log_probs = torch.nn.functional.softmax(output.logits, dim=0)
    neg_log_probs.append(log_probs)


[tensor([[[1., 1., 1.,  ..., 1., 1., 1.],
          [1., 1., 1.,  ..., 1., 1., 1.],
          [1., 1., 1.,  ..., 1., 1., 1.],
          ...,
          [1., 1., 1.,  ..., 1., 1., 1.],
          [1., 1., 1.,  ..., 1., 1., 1.],
          [1., 1., 1.,  ..., 1., 1., 1.]]]),
 tensor([[[1., 1., 1.,  ..., 1., 1., 1.],
          [1., 1., 1.,  ..., 1., 1., 1.],
          [1., 1., 1.,  ..., 1., 1., 1.],
          ...,
          [1., 1., 1.,  ..., 1., 1., 1.],
          [1., 1., 1.,  ..., 1., 1., 1.],
          [1., 1., 1.,  ..., 1., 1., 1.]]]),
 tensor([[[1., 1., 1.,  ..., 1., 1., 1.],
          [1., 1., 1.,  ..., 1., 1., 1.],
          [1., 1., 1.,  ..., 1., 1., 1.],
          ...,
          [1., 1., 1.,  ..., 1., 1., 1.],
          [1., 1., 1.,  ..., 1., 1., 1.],
          [1., 1., 1.,  ..., 1., 1., 1.]]]),
 tensor([[[1., 1., 1.,  ..., 1., 1., 1.],
          [1., 1., 1.,  ..., 1., 1., 1.],
          [1., 1., 1.,  ..., 1., 1., 1.],
          ...,
          [1., 1., 1.,  ..., 1., 1., 1.],
       

# Likelihood